### Text-Sampling des Gesamtkorpus des Deutschen Textarchiv ###

Dieses Jupyter-Notebook wurde eingerichtet, um die Einzelschritte für das Text-Sampling der Texte aufzuzeigen, und für die Gewährleistung der Nachprüfbarkeit. Es sollte ein interaktives Herausfinden von wichtigen Erkenntnissen ermöglichen.

Für die Datenbank-Analyse werden folgende libraries benötigt (nicht komplett):

- Pandas, für Visualisierungen
- SQLite, für Datenbankerstellung und -abfragen
- NLTK, für Wortdistributionen
- Matplotlib, für Visualisierungen
- Wordcloud, für Tokenfrequenz
- evtl. Plotly, für interaktive Visualisierungen
- 
- ... und mehr (extensible)

Vorgehen für das Sampling (mit Hilfe von ChatGPT, Prompt: "Any tips on sampling?")


1. Analyse der proportionalen Repräsentationen von Dekaden, Genres, Länge. Ziel: Erkenntnis über die Sample-Menge für Dekaden, Genres, Länge in Bezug auf ihrer prozentualen Repräsentation im Gesamtkorpus. (Geschichtete Zufallsstichprobe)
2. Zufallsstichprobe mit Einschränkungen (Automatisiert):
    - repräsentative Menge an Beispieltexten in Bezug auf Gesamtkorpus, Länge (sehr kurze und sehr lange)
    - "Handverlesene", sehr repräsentative Texte kriegen Vorrang bei der Auswahl
3. Quirks-Analyse:
    - Unüblich oft vorkommende Tokens
    - Archaischer oder genre-spezifischer Wortschatz
    - N-Gram Analyse
    - Tokenlänge-Distribution
    - Buchstaben-Distribution
    - Verwendung von weiteren Sprachen (z.B. Latein)
Für diesen Schritt sollte möglichst mit Visualisierungen gearbeitet werden: Word Clouds, Bar charts, top-N Wörter, heatmaps, Histogramme, Scatterplots, Time-Series Plots
4. Manuelle Inspektionen


## 1. Analysen der proportionalen Repräsentationen von Dekaden, Genres, Länge ##

***Dekaden***

Zuerst werden die Jahre ausgelesen von der Datenbank und in ein DataFrame gesetzt (vereinfachte Visualisierungen)

In [2]:
import sqlite3
import pandas as pd

con = sqlite3.connect("Datenbank.db")
abfrage = """SELECT publikationsjahr FROM my_data;"""

df = pd.read_sql_query(abfrage, con)

con.close()

Nun werden die Texte pro Jahr gezählt mit pandas:

In [3]:
yearly_count = df.groupby("publikationsjahr").size().reset_index(name="count")

Hier werden die Daten dann visualisiert mithilfe von plotly. Zuerst muss plotly aber installiert werden.

In [1]:
%pip install plotly

  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
Using cached plotly-5.24.1-py3-none-any.whl (19.1 MB)
Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Nun kann Plotly für die Visualisierung verwendet werden:

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Bar(name="Texte pro Jahr", x=yearly_count["publikationsjahr"], y=yearly_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Jahr",
    xaxis_title="Jahr",
    yaxis_title="Anzahl",
    hovermode="x"
)

fig.show()


***Analyse:***

Sehr auffällig ist der Spike in den Jahren von 1840-1848. Dies fällt in den Zeitraum verschiedener Revolutionen in Europa und muss separat berücksichtigt werden beim Sampling. Es wurde entschieden, eine Stratifizierung vorzunehmen, in Epochen, um ein proportionales Sampling zu ermöglichen. Dies sollte mehr Einblick geben, wie sich die Verteilung der Texte innerhalb (mehr oder weniger arbiträren) Epochen verändert hat. 

Die Epochen werden aufgeteilt in:
- Spätmittelalter (1473-1500)
- Renaissance (1501-1600)
- Barock (1601-1700)
- Aufklärung (1701-1800)
- Frühes 19. Jahrhundert (1801-1839)
- Revolutionsjahre (1840-1850)
- Industrielle Revolution (1851-1900)
- Moderne (1901-1987)

Dafür wird eine neue Spalte "Zeitstratum" in der SQL-Datenbank erstellt und die Texte gemäss ihrem Publikationsjahr eingestuft.

In [3]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

cur.execute("""
    ALTER TABLE my_data
    ADD COLUMN 'zeitstratum'
""")

cur.execute("""
    UPDATE my_data
    SET zeitstratum = CASE
            WHEN publikationsjahr BETWEEN 1473 AND 1500 THEN 'Spätmittelalter'
            WHEN publikationsjahr BETWEEN 1501 AND 1600 THEN 'Renaissance'
            WHEN publikationsjahr BETWEEN 1601 AND 1700 THEN 'Barock'
            WHEN publikationsjahr BETWEEN 1701 AND 1800 THEN 'Aufklärung'
            WHEN publikationsjahr BETWEEN 1801 AND 1839 THEN 'Frühes 19. Jahrhundert'
            WHEN publikationsjahr BETWEEN 1840 AND 1850 THEN 'Revolutionsjahre'
            WHEN publikationsjahr BETWEEN 1851 AND 1900 THEN 'Industrielle Revolution'
            ELSE 'Moderne'
    END;
""")
con.commit()


Tests:

In [ ]:
testjahre = [1473, 1536, 1650, 1750, 1830, 1848, 1890, 1914] # Diese Daten haben mindestens einen Text (siehe Visualisierung oben)

for year in testjahre:
    print(f"Das Jahr {year} ist in der Epoche {cur.execute("SELECT zeitstratum FROM my_data WHERE publikationsjahr=?", (year,)).fetchone()[0]}")

In [5]:
con.close()

Nun kann eine neue Visualisierung gemäss diesen Epochen gemacht werden, in der Form einer Bar-Chart.

In [ ]:
import plotly.graph_objects as go
import sqlite3
import pandas as pd

con = sqlite3.connect("Datenbank.db")
query = """SELECT zeitstratum FROM my_data"""

df = pd.read_sql_query(query, con)

con.close()

epoch_count = df.groupby("zeitstratum").size().reset_index(name="count")

fig = go.Figure(data=[go.Bar(name="Texte pro Epoche", x=epoch_count["zeitstratum"], y=epoch_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Epoche",
    xaxis_title="Epoche",
    yaxis_title="Anzahl",
    hovermode="x"
)

fig.show()


***Hauptgenre***

Äquivalent zur vorherigen Abfragen, deshalb abgekürzt in einer Ausführung:

In [ ]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

con = sqlite3.connect("Datenbank.db")
abfrage = """SELECT hauptklasse FROM my_data"""

df = pd.read_sql_query(abfrage, con)

con.close()

class_count = df.groupby("hauptklasse").size().reset_index(name="count")

fig = go.Figure(data=[go.Bar(name="Texte pro Hauptklasse", x=class_count["hauptklasse"], y=class_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Hauptklasse",
    xaxis_title="Hauptklasse",
    yaxis_title="Anzahl",
    hovermode="x"
)

fig.show()

***Analyse***

Da die Texte in nur vier Hauptklassen unterteilt werden, ist diese Visualisierung nicht wirklich repräsentativ. Die Subgenres geben wahrscheinlich mehr Aufschluss über die Verteilung.

***Subgenre***

In [ ]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

con = sqlite3.connect("Datenbank.db")
abfrage = """SELECT subklasse FROM my_data"""

df = pd.read_sql_query(abfrage, con)

con.close()

class_count = df.groupby("subklasse").size().reset_index(name="count")

fig = go.Figure(data=[go.Bar(name="Texte pro Subgenre", x=class_count["subklasse"], y=class_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Subgenre",
    xaxis_title="Subgenre",
    yaxis_title="Anzahl",
    hovermode="x"
)

fig.show()

***Analyse***

Problematisch an dieser Visualisierung ist die grosse Anzahl an Subgenres: 127. Um den Noise der Visualisierung zu verringern, werden die Hauptgenres und Subgenres kombiniert. Zunächst wird ein Dictionary erstellt, dass die Subgenres als Teilmenge der Hauptgenres auszeichnet.

In [14]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

genre_dict = {
    "Belletristik" : [],
    "Gebrauchsliteratur" : [],
    "Wissenschaft" : [],
    "Zeitung" : []
}

for genre in genre_dict:
    list_of_rows = cur.execute("SELECT DISTINCT subklasse FROM my_data WHERE hauptklasse=?", (genre, ))
    list_of_subgenres = [row[0] for row in list_of_rows]
    genre_dict[genre] = list_of_subgenres

In [ ]:
import pprint as pp
pp.pprint(genre_dict)

In [ ]:
print(len(genre_dict["Wissenschaft"]))

wissenschaft = "Medizin, Physik, Biologie, Zoologie, Anthropologie, Geographie; Ethnologie, Mathematik, Astronomie, Geologie, Naturgeschichte, Chemie, Anatomie, Physiologie, Tierheilkunde, Gartenbau, Landwirtschaft. Sprachwissenschaft, Literaturwissenschaft, Theologie, Philosophie, Geschichte, Altertumskunde, Archäologie, Ethnologie, Philologie. Psychologie, Soziologie, Politik, Gesellschaft, Gesellschaftswissenschaften, Kameralwissenschaft, Militär. Ökonomie, Recht, Jura, Politik, Ökonomie, Gesellschaftswissenschaften; Recht, Politik, Anstandsliteratur. Technik, Musik, Musikwissenschaft, Kunstgeschichte, Buchwesen, Architektur, Bergbau, Glasherstellung. Geographie, Geowissenschaften, Naturwissenschaft, Naturwissenschaft; Philosophie, Alchemie, Alchemie, Medizin, Sonstiges, Historiographie, Historiographie; Recht, Ordensliteratur:Jesuiten. Pädagogik, Brief, Astronomie "
missing = []

for subgenre in genre_dict["Wissenschaft"]:
    if subgenre not in wissenschaft:
        missing.append(subgenre)

print(missing)

Mit Hilfe von ChatGPT kann diese Hauptgenres-Subgenres-Klassifizierung in eine übersichtlichere Gruppierung aufgeteilt werden (Prompt: "Wie können diese Hauptgenres und Subgenres repräsentativ verknappt oder zusammengefügt werden, damit die Anzahl Genres übersichtlich bleibt, gib mir einen Vorschlag" + genre_dict). Der Output von ChatGPT sieht diese Stratifizierung vor:

- Belletristik:
    - Lyrik & Poesie
        - Lyrik
        - Lyrik;Libretto
        - Lyrik;Drama
        - Lyrik;Epigramm
        - Lyrik;Prosa;Drama
        - Reimpaarspruch
        - Lied
        - Satire
        - Städtelob
        - Religiöse Reimpaarerzählung
        - Lyrik;Prosa
        - Epik
        - Heldenepik
    - Prosa & Erzählungen
        - Roman
        - Novelle
        - Prosa
        - Epos
        - Dialog
        - Briefroman
        - Schäferdichtung
        - Reiseliteratur
    - Drama
        - Drama
        - Drama;Prosa
        - Lyrik;Drama;Prosa
    - Autobiographische & Biographische Werke
        - Autobiographie
        - Biographie
        - (Auto)biographie
    - Kinder- & Jugendliteratur
        - Kinderliteratur
        - Märchen
    - Sonstige literarische Genres
        - Briefe
        - Lexikon

- Gebrauchsliteratur:
    - Praktische Literatur
        - Hausväterliteratur
        - Kochbuch
        - Handbuch
        - Rhetorik
        - Lexikon
        - Wörterbuch
        - Briefsteller
        - Rechnungsbuch
        - Gartenbau
        - Pädagogik
        - Schulbuch
    - Religiöse & Erbauliche Literatur
        - Leichenpredigt
        - Gelegenheitsschrift;Tod
        - Gelegenheitsschrift;Tod;Lyrik
        - Erbauungsliteratur
        - Andachtsbuch
        - Bibelübersetzung
        - Gelegenheitsschrift;Fest
        - Anstandsliteratur
    - Politische & Rechtliche Texte
        - Politik
        - Verordnung
        - Reformschrift
        - Recht
        - Bericht
        - Streitschrift;Brief
        - Amtsdruckschrift
        - Gelegenheitsschrift;Vertrag
    - Wissenschaftlich-Populäre Literatur
        - Populärwissenschaft
        - Philologie
        - Zoologie
        - Biologie
        - Naturwissenschaft
        - Astronomie
        - Mathematik
        - Geologie
        - Bergbau
        - Psychologie
        - Astrologie
        - Pflanzenbuch
        - Kriminalistik
        - Theologie
        - Technik
        - Ökonomie
        - Philosophie
        - Medizin
    - Künstlerische & Kreative Texte
        - Kunst
        - Musik
        - Verslehre
        - Poetik
        - Zeitschrift
        - Satire
        - Rede
    - Reise- & Erfahrungsberichte
        - Reiseliteratur
        - Tagebuch
        - Autobiographie
        - Rezension
    - Sonstige Gebrauchsliteratur
        - Gesellschaft
        - Regionalgeschichte
        - Kolportageliteratur
        - Sport
        - Flugschrift
        - Sonstiges
- Wissenschaft:
    - Naturwissenschaften
        - Medizin
        - Physik
        - Biologie
        - Zoologie
        - Astronomie
        - Geologie
        - Naturgeschichte
        - Chemie
        - Anatomie
        - Physiologie
        - Tierheilkunde
        - Geographie
        - Geowissenschaft
        - Naturwissenschaft
        - Mathematik
        - Anthropologie
    - Geisteswissenschaften
        - Sprachwissenschaft
        - Literaturwissenschaft
        - Theologie
        - Philosophie
        - Gesschichte
        - Altertumskunde
        - Archäologie
        - Ethnologie
        - Philologie
        - Musik
        - Musikwissenschaft
        - Kunstgeschichte
        - Architektur
        - Naturwissenschaft;Philosophie
        - Historiographie
        - Pädagogik
        - Geographie;Ethnologie
    - Sozialwissenschaften
        - Psychologie
        - Soziologie
        - Politik
        - Gesellschaft
        - Gesellschaftswissenschaften
        - Kameralwissenschaft
        - Militär
    - Wirtschaft und Recht
        - Ökonomie
        - Recht
        - Jura
        - Politik
        - Gesellschaftswissenschaften;Recht
        - Politik
        - Anstandsliteratur
        - Historiographie;Recht
    - Technische Wissenschaften
        - Technik
        - Bergbau
        - Glasherstellung
        - Landwirtschaft
        - Gartenbau
    - Sonstiges
        - Alchemie
        - Sonstiges
        - Ordensliteratur:Jesuiten
        - Brief

- Zeitung



Fine-Tuning war noch nötig, aber schliesslich gibt es diese Subgenres und Hauptgenres:
- Belletristik
    - Lyrik & Poesie
    - Prosa & Erzählungen
    - Drama
    - Autobiographische & Biographische Werke
    - Kinder- & Jugendliteratur
    - Sonstige literarische Genres
- Gebrauchsliteratur
    - Praktische Literatur
    - Religiöse & Erbauliche Literatur
    - Politische & Rechtliche Texte
    - Wissenschaftlich-Populäre Literatur
    - Künstlerische & Kreative Texte
    - Reise- & Erfahrungsberichte
    - Sonstige Gebrauchsliteratur
- Wissenschaften
    - Naturwissenschaften
    - Geisteswissenschaften
    - Sozialwissenschaften
    - Wirtschaft und Recht
    - Technische Wissenschaften
    - Sonstiges
- Zeitung

In einem nächsten Schritt werden diese Genres in eine neue Spalte "Genrestratum" in die SQL-Datenbank eingespeist. Dieser Code ist ein bisschen lang, aufgrund der Fülle an Subgenres

In [ ]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

cur.execute("""
            ALTER TABLE my_data
            ADD COLUMN 'genrestratum' TEXT
            """)

In [37]:
con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

cur.execute("""
    UPDATE my_data
    SET genrestratum = CASE
            WHEN hauptklasse IN ('Belletristik') AND subklasse IN (
                'Lyrik',
                'Lyrik, Libretto',
                'Lyrik, Drama',
                'Lyrik, Epigramm',
                'Lyrik; Prosa; Drama',
                'Reimpaarspruch',
                'Lied',
                'Satire',
                'Städtelob',
                'Religiöse Reimpaarerzählung',
                'Lyrik; Prosa',
                'Epik',
                'Heldenepik'
            ) THEN 'Belletristik; Lyrik,Poesie'
            WHEN hauptklasse IN ('Belletristik') AND subklasse IN (
                'Roman',
                'Novelle',
                'Prosa',
                'Epos',
                'Dialog',
                'Briefroman',
                'Schäferdichtung',
                'Reiseliteratur'
            ) THEN 'Belletristik; Prosa,Erzählungen'
            WHEN hauptklasse IN ('Belletristik') AND subklasse IN (
                'Drama',
                'Drama; Prosa',
                'Lyrik; Drama; Prosa'
            ) THEN 'Belletristik; Drama'
            WHEN hauptklasse IN ('Belletristik') AND subklasse IN (
                'Autobiographie',
                'Biographie',
                '(Auto)biographie'
            ) THEN 'Belletristik; Autobiographische und Biographische Werke'
            WHEN hauptklasse IN ('Belletristik') AND subklasse IN (
                'Kinderliteratur',
                'Märchen'
            ) THEN 'Belletristik; Kinderliteratur'
            WHEN hauptklasse IN ('Belletristik') AND subklasse IN (
                'Briefe',
                'Lexikon'
            ) THEN 'Belletristik; Sonstige'
            WHEN hauptklasse IN ('Gebrauchsliteratur') AND subklasse IN (
                'Hausväterliteratur',
                'Kochbuch',
                'Handbuch',
                'Rhetorik',
                'Lexikon',
                'Wörterbuch',
                'Briefsteller',
                'Rechnungsbuch',
                'Gartenbau',
                'Pädagogik',
                'Schulbuch',
                'Handbuch, Rhetorik'
            ) THEN 'Gebrauchsliteratur; Praktische Literatur'
            WHEN hauptklasse IN ('Gebrauchsliteratur') AND subklasse IN (
                'Leichenpredigt',
                'Gelegenheitsschrift:Tod',
                'Gelegenheitsschrift:Tod; Lyrik',
                'Erbauungsliteratur',
                'Andachtsbuch',
                'Bibelübersetzung',
                'Gelegenheitsschrift:Fest',
                'Anstandsliteratur'
            ) THEN 'Gebrauchsliteratur; Religiöse und Erbauliche Literatur'
            WHEN hauptklasse IN ('Gebrauchsliteratur') AND subklasse IN (
                'Politik',
                'Verordnung',
                'Reformschrift',
                'Recht',
                'Bericht',
                'Streitschrift; Brief',
                'Amtsdruckschrift',
                'Gelegenheitsschrift:Vertrag'
            ) THEN 'Gebrauchsliteratur; Politische und Rechtliche Texte'
            WHEN hauptklasse IN ('Gebrauchsliteratur') AND subklasse IN (
                'Populärwissenschaft',
                'Philologie',
                'Zoologie',
                'Biologie',
                'Naturwissenschaft',
                'Astronomie',
                'Mathematik',
                'Geologie',
                'Bergbau',
                'Psychologie',
                'Astrologie',
                'Pflanzenbuch',
                'Kriminalistik',
                'Theologie',
                'Technik',
                'Ökonomie',
                'Philosophie',
                'Medizin'
            ) THEN 'Gebrauchsliteratur; Wissenschaftlich-Populäre Literatur'
            WHEN hauptklasse IN ('Gebrauchsliteratur') AND subklasse IN (
                'Kunst',
                'Musik',
                'Verslehre',
                'Poetik',
                'Zeitschrift',
                'Satire',
                'Rede'
            ) THEN 'Gebrauchsliteratur; Künstlerische und Kreative Texte'
            WHEN hauptklasse IN ('Gebrauchsliteratur') AND subklasse IN (
                'Reiseliteratur',
                'Tagebuch',
                'Autobiographie',
                'Rezension'
            ) THEN 'Gebrauchsliteratur; Reise- und Erfahrungsberichte'
            WHEN hauptklasse IN ('Gebrauchsliteratur') AND subklasse IN (
                'Gesellschaft',
                'Regionalgeschichte',
                'Kolportageliteratur',
                'Sport',
                'Flugschrift',
                'Sonstiges',
                'Brief'
            ) THEN 'Gebrauchsliteratur; Sonstige'
            WHEN hauptklasse IN ('Wissenschaft') AND subklasse IN (
                'Medizin',
                'Physik',
                'Biologie',
                'Zoologie',
                'Astronomie',
                'Geologie',
                'Naturgeschichte',
                'Chemie',
                'Anatomie',
                'Physiologie',
                'Tierheilkunde',
                'Geographie',
                'Geowissenschaft',
                'Naturwissenschaft',
                'Mathematik',
                'Anthropologie',
                'Geowissenschaften'
            ) THEN 'Wissenschaft; Naturwissenschaften'
            WHEN hauptklasse IN ('Wissenschaft') AND subklasse IN (
                'Sprachwissenschaft',
                'Literaturwissenschaft',
                'Theologie',
                'Philosophie',
                'Geschichte',
                'Altertumskunde',
                'Archäologie',
                'Ethnologie',
                'Philologie',
                'Musik',
                'Musikwissenschaft',
                'Kunstgeschichte',
                'Architektur',
                'Naturwissenschaft; Philosophie',
                'Historiographie',
                'Pädagogik',
                'Geographie; Ethnologie'
            ) THEN 'Wissenschaft; Geisteswissenschaften'
            WHEN hauptklasse IN ('Wissenschaft') AND subklasse IN (
                'Psychologie',
                'Soziologie',
                'Politik',
                'Gesellschaft',
                'Gesellschaftswissenschaften',
                'Kameralwissenschaft',
                'Militär'
            ) THEN 'Wissenschaft; Sozialwissenschaften'
            WHEN hauptklasse IN ('Wissenschaft') AND subklasse IN (
                'Ökonomie',
                'Recht',
                'Jura',
                'Politik',
                'Gesellschaftswissenschaften; Recht',
                'Politik',
                'Anstandsliteratur',
                'Historiographie; Recht',
                'Buchwesen',
                'Politik, Anstandsliteratur',
                'Politik, Ökonomie'
            ) THEN 'Wissenschaft; Wirtschaft und Recht'
            WHEN hauptklasse IN ('Wissenschaft') AND subklasse IN (
                'Technik',
                'Bergbau',
                'Glasherstellung',
                'Landwirtschaft',
                'Gartenbau'
            ) THEN 'Wissenschaft; Technische Wissenschaften'
            WHEN hauptklasse IN ('Wissenschaft') AND subklasse IN (
                'Alchemie',
                'Sonstiges',
                'Ordensliteratur:Jesuiten',
                'Brief',
                'Alchemie, Medizin'
            ) THEN 'Wissenschaft; Sonstiges'
            WHEN hauptklasse IN ('Zeitung') THEN 'Zeitung'

    END
""")

con.commit()

Hier folgt ein Test, um sicherzustellen, dass die Zuweisung für jeden Text erfolgreich war:

In [ ]:
# Schauen, ob alle Texte einen Eintrag in der genrestratum-Spalte haben
without_genrestratum = cur.execute("SELECT text_index FROM my_data WHERE genrestratum=NULL").fetchall()

if not without_genrestratum:
    print("Alles zugewiesen!")
else:
    print(without_genrestratum)

In [38]:
con.close()

Nun können die Genres visualisiert werden! Zunächst einmal die Verteilung über alle Genrestrata, dann spezifisch innerhalb der Hauptklassen.

In [ ]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

con = sqlite3.connect("Datenbank.db")
query = """SELECT genrestratum FROM my_data"""

df = pd.read_sql_query(query, con)

genre_count = df.groupby("genrestratum").size().reset_index(name="count")

fig = go.Figure(data=[go.Bar(name="Texte pro Genre", x=genre_count["genrestratum"], y=genre_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Genre",
    xaxis_title="Genres",
    yaxis_title="Anzahl Texte",
    hovermode="x"
)

fig.show()

Belletristik:

In [ ]:
query_belletristik = """SELECT genrestratum FROM my_data WHERE hauptklasse='Belletristik'"""

df = pd.read_sql_query(query_belletristik, con)

belletristik_count = df.groupby("genrestratum").size().reset_index(name="count")

fig = go.Figure(data=[go.Bar(name="Texte pro Genre in Belletristik", x=belletristik_count["genrestratum"], y=belletristik_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Subgenre in Belletristik",
    xaxis_title="Subgenres",
    yaxis_title="Anzahl Texte",
    hovermode="x"
)
fig.show()

Gebrauchsliteratur:

In [ ]:
query_gebrauchsliteratur = """SELECT genrestratum FROM my_data WHERE hauptklasse='Gebrauchsliteratur'"""

df = pd.read_sql_query(query_gebrauchsliteratur, con)

gebrauchsliteratur_count = df.groupby("genrestratum").size().reset_index(name="count")

fig = go.Figure(data=[go.Bar(name="Texte pro Genre in Gebrauchsliteratur", x=gebrauchsliteratur_count["genrestratum"], y=gebrauchsliteratur_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Subgenre in Gebrauchsliteratur",
    xaxis_title="Subgenres",
    yaxis_title="Anzahl Texte",
    hovermode="x"
)
fig.show()

Wissenschaft:

In [ ]:
query_wissenschaft = """SELECT genrestratum FROM my_data WHERE hauptklasse='Wissenschaft'"""

df = pd.read_sql_query(query_wissenschaft, con)

wissenschaft_count = df.groupby("genrestratum").size().reset_index(name="count")

fig = go.Figure(data=[go.Bar(name="Texte pro Genre in Wissenschaft", x=wissenschaft_count["genrestratum"], y=wissenschaft_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Subgenre in Wissenschaft",
    xaxis_title="Subgenres",
    yaxis_title="Anzahl Texte",
    hovermode="x"
)
fig.show()

## Kombinierte Strata Epochen x Hauptklasse ##
An dieser Stelle soll nun eine Kombination stattfinden zwischen den vorhin definierten Epochenstrata und den Hauptklassenstrata. So kann gewährleistet werden, dass eine repräsentative Auswahl an Stichprobentexten gemacht werden kann. 

Die Subgenres wurden weggelassen, wegen der erhöhten Komplexität. Durch die Kombination von Epochen und Hauptklassen ergeben sich:

Epochen x Hauptklasse = 8 * 4 = 32 potentielle Kombinationen.

Mit den Subgenres erhöht sich diese Zahl um:

Epochen x Hauptklasse x Subklasse = 8 * 4 * 20 = 640 potentielle Kombinationen. 

Von diesen 640 Kombinationen werden voraussichtlich ziemlich viele leer sein (z.B. Spätmittelalter x Wissenschaft x Philosophie) aufgrund der Sparsity der Daten. 

Die Hauptdimensionen Epoche und Hauptklasse sind für diese Analyse repräsentativ genug.

Zunächst wird eine neue Spalte erstellt mit dem Titel "epochen-genres-strata":

In [46]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

cur.execute("""
        ALTER TABLE my_data
        ADD epochen_genres_strata TEXT
""")

con.commit()
con.close()

Um die Spalte mit den Epoche_Genres_strata zu füllen, wird bei jeder Reihe geprüft, welcher Epoche sie angehört und was die hauptklasse ist.

In [48]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

cur.execute("""
    UPDATE my_data
    SET epochen_genres_strata = CASE
            WHEN zeitstratum = 'Spätmittelalter' AND hauptklasse = 'Belletristik' THEN 'Spätmittelalter x Belletristik'
            WHEN zeitstratum = 'Spätmittelalter' AND hauptklasse = 'Gebrauchsliteratur' THEN 'Spätmittelalter x Gebrauchsliteratur'
            WHEN zeitstratum = 'Spätmittelalter' AND hauptklasse = 'Wissenschaft' THEN 'Spätmittelalter x Wissenschaft'
            WHEN zeitstratum = 'Spätmittelalter' AND hauptklasse = 'Zeitung' THEN 'Spätmittelalter x Zeitung'
            WHEN zeitstratum = 'Barock' AND hauptklasse = 'Belletristik' THEN 'Barock x Belletristik'
            WHEN zeitstratum = 'Barock' AND hauptklasse = 'Gebrauchsliteratur' THEN 'Barock x Gebrauchsliteratur'
            WHEN zeitstratum = 'Barock' AND hauptklasse = 'Wissenschaft' THEN 'Barock x Wissenschaft'
            WHEN zeitstratum = 'Barock' AND hauptklasse = 'Zeitung' THEN 'Barock x Zeitung'
            WHEN zeitstratum = 'Renaissance' AND hauptklasse = 'Belletristik' THEN 'Renaissance x Belletristik'
            WHEN zeitstratum = 'Renaissance' AND hauptklasse = 'Gebrauchsliteratur' THEN 'Renaissance x Gebrauchsliteratur'
            WHEN zeitstratum = 'Renaissance' AND hauptklasse = 'Wissenschaft' THEN 'Renaissance x Wissenschaft'
            WHEN zeitstratum = 'Renaissance' AND hauptklasse = 'Zeitung' THEN 'Renaissance x Zeitung'
            WHEN zeitstratum = 'Aufklärung' AND hauptklasse = 'Belletristik' THEN 'Aufklärung x Belletristik'
            WHEN zeitstratum = 'Aufklärung' AND hauptklasse = 'Gebrauchsliteratur' THEN 'Aufklärung x Gebrauchsliteratur'
            WHEN zeitstratum = 'Aufklärung' AND hauptklasse = 'Wissenschaft' THEN 'Aufklärung x Wissenschaft'
            WHEN zeitstratum = 'Aufklärung' AND hauptklasse = 'Zeitung' THEN 'Aufklärung x Zeitung'
            WHEN zeitstratum = 'Frühes 19. Jahrhundert' AND hauptklasse = 'Belletristik' THEN 'Frühes 19. Jahrhundert x Belletristik'
            WHEN zeitstratum = 'Frühes 19. Jahrhundert' AND hauptklasse = 'Gebrauchsliteratur' THEN 'Frühes 19. Jahrhundert x Gebrauchsliteratur'
            WHEN zeitstratum = 'Frühes 19. Jahrhundert' AND hauptklasse = 'Wissenschaft' THEN 'Frühes 19. Jahrhundert x Wissenschaft'
            WHEN zeitstratum = 'Frühes 19. Jahrhundert' AND hauptklasse = 'Zeitung' THEN 'Frühes 19. Jahrhundert x Zeitung'
            WHEN zeitstratum = 'Revolutionsjahre' AND hauptklasse = 'Belletristik' THEN 'Revolutionsjahre x Belletristik'
            WHEN zeitstratum = 'Revolutionsjahre' AND hauptklasse = 'Gebrauchsliteratur' THEN 'Revolutionsjahre x Gebrauchsliteratur'
            WHEN zeitstratum = 'Revolutionsjahre' AND hauptklasse = 'Wissenschaft' THEN 'Revolutionsjahre x Wissenschaft'
            WHEN zeitstratum = 'Revolutionsjahre' AND hauptklasse = 'Zeitung' THEN 'Revolutionsjahre x Zeitung'
            WHEN zeitstratum = 'Industrielle Revolution' AND hauptklasse = 'Belletristik' THEN 'Industrielle Revolution x Belletristik'
            WHEN zeitstratum = 'Industrielle Revolution' AND hauptklasse = 'Gebrauchsliteratur' THEN 'Industrielle Revolution x Gebrauchsliteratur'
            WHEN zeitstratum = 'Industrielle Revolution' AND hauptklasse = 'Wissenschaft' THEN 'Industrielle Revolution x Wissenschaft'
            WHEN zeitstratum = 'Industrielle Revolution' AND hauptklasse = 'Zeitung' THEN 'Industrielle Revolution x Zeitung'
            WHEN zeitstratum = 'Moderne' AND hauptklasse = 'Belletristik' THEN 'Moderne x Belletristik'
            WHEN zeitstratum = 'Moderne' AND hauptklasse = 'Gebrauchsliteratur' THEN 'Moderne x Gebrauchsliteratur'
            WHEN zeitstratum = 'Moderne' AND hauptklasse = 'Wissenschaft' THEN 'Moderne x Wissenschaft'
            WHEN zeitstratum = 'Moderne' AND hauptklasse = 'Zeitung' THEN 'Moderne x Zeitung'
    END
""")

con.commit()
con.close()

Nun visualisieren wir diese neue Klassifizierung. Sie bildet die Grundlage für das Sampling.

In [2]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

con = sqlite3.connect("Datenbank.db")
query = """SELECT epochen_genres_strata FROM my_data"""

df = pd.read_sql_query(query, con)

epochen_genres_count = df.groupby("epochen_genres_strata").size().reset_index(name="count")

fig = go.Figure(data=[go.Bar(name="Anzahl Texte als Epochen x Hauptklasse", x=epochen_genres_count["epochen_genres_strata"], y=epochen_genres_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Epochen x Hauptklasse",
    xaxis_title="Epoche x Hauptklasse",
    yaxis_title="Anzahl Texte",
    hovermode="x"
)

fig.show()

con.close()

Jetzt wird die Berechnung der Gewichtung gemacht, um Herauszufinden, welche Samplingmenge von wo genommen werden sollte. Dafür kann das DataFrame von vorhin verwendet werden. Die Gewichtung und die Samplesize müssen noch hinzugefügt werden.

In [4]:
epochen_genres_count["weight"] = [count / 4436 for count in epochen_genres_count["count"]]
epochen_genres_count["sample"] = [weight * 100 for weight in epochen_genres_count["weight"]]

fig = go.Figure(data=[go.Table(
    header=dict(values=["Epoche x Hauptklasse", "Anzahl", "Gewichtung (Anzahl / 4436)", "Samplezahl (Gewichtung * 100)"],
                fill_color="paleturquoise",
                align="left"),
    cells=dict(values=[list(epochen_genres_count["epochen_genres_strata"]), list(epochen_genres_count["count"]), list(round(epochen_genres_count["weight"], 4)), list(round(epochen_genres_count["sample"], 0))],
               fill_color="lavender",
               align="left"))
])
fig.update_layout(height=1490)
fig.show()

Wie erwartet sind einige der Epoche x Hauptklasse-Elemente nur in geringer Anzahl vertreten. Im Detail sind das:
- Barock x Zeitung (Anzahl 5)
- Renaissance x Belletristik (Anzahl 2)
- Renaissance x Wissenschaft (Anzahl 2)
- Renaissance x Zeitung (Anzahl 1)
- Spätmittelalter x Belletristik (Anzahl 4)
- Spätmittelalter x Gebrauchsliteratur (Anzahl 12)

Für die Repräsentation dieser seltenen Strata wird jeweils ein Sample genommen. Um die Gesamtzahl der Samples bei 100 zu lassen, werden vier Samples von dem dominanten Stratum Revolutionsjahre x Zeitung und 2 von dem Stratum Barock x Gebrauchsliteratur abgezogen. 

Die ajustierte Samplemenge sieht nun so aus:

In [5]:
epochen_genres_count["adjusted_sample"] = [sample if round(sample, 0) > 0 else sample + 1 for sample in epochen_genres_count["sample"]]
epochen_genres_count["adjusted_sample"][5] -= 2 # Barock x Gebrauchsliteratur
epochen_genres_count["adjusted_sample"][27] -= 4 # Revolutionsjahre x Zeitung

fig = go.Figure(data=[go.Table(
    header=dict(values=["Epoche x Hauptklasse", "Anzahl", "Gewichtung (Anzahl / 4436)", "Samplezahl (Gewichtung * 100)", "Ajustierte Samplezahl"],
                fill_color="paleturquoise",
                align="left"),
    cells=dict(values=[list(epochen_genres_count["epochen_genres_strata"]), list(epochen_genres_count["count"]), list(round(epochen_genres_count["weight"], 4)), list(round(epochen_genres_count["sample"], 0)), list(round(epochen_genres_count["adjusted_sample"], 0))],
               fill_color="lavender",
               align="left"))
])
fig.update_layout(height=1800)
fig.show()

C:\Users\raoul\AppData\Local\Temp\ipykernel_27716\3234172964.py:2: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\raoul\AppData\Local\Temp\ipykernel_27716\3234172964.py:2: SettingWithCopyWarning:


A value is trying to be set on a co

### 2. Extrahieren der Samples und Kreieren eines neuen Tables innerhalb der SQL-Datenbank ###
In diesem Schritt wird die gewichtete Zufallsstichprobe entnommen und in eine neue table "samples" in der Datenbank eingespiesen.

**WICHTIG: Die Struktur der Datenbank wird verändert, ein Backup sollte lokal kreiert werden.**

In [101]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

cur.execute(
    """
    CREATE TABLE samples (
        text_index INTEGER PRIMARY KEY,
        haupttitel TEXT,
        untertitel TEXT,
        volumetitel TEXT,
        autor TEXT,
        hauptklasse TEXT,
        subklasse TEXT,
        sprache TEXT,
        text TEXT,
        publikationsjahr INTEGER,
        zeitstratum TEXT,
        genrestratum TEXT,
        epochen_genres_strata TEXT
    );
    """
)
con.commit()
con.close()

In [127]:
con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

epoch_list = list(epochen_genres_count["epochen_genres_strata"])
sample_list = list(map(int, round(epochen_genres_count["adjusted_sample"], 0)))

epoch_samples = dict(zip(epoch_list, sample_list))

for epochexgenre, sample_size in epoch_samples.items():
    sample_size = int(sample_size)
    query = f"""
        INSERT INTO samples (text_index, haupttitel, untertitel, volumetitel, autor, hauptklasse, subklasse, sprache, text, publikationsjahr, zeitstratum, genrestratum, epochen_genres_strata)
                SELECT text_index, haupttitel, untertitel, volumetitel, autor, hauptklasse, subklasse, sprache, text, publikationsjahr, zeitstratum, genrestratum, epochen_genres_strata
                FROM my_data
                WHERE epochen_genres_strata = ?
                ORDER BY RANDOM()
                LIMIT {sample_size}

"""
    cur.execute(query, (epochexgenre,))

con.commit()
con.close()

Jetzt wurde die samples-Datenbank befüllt mit 100 zufälligen Textstichproben. An dieser Stelle werden nun die Titel und Untertitel, der Autor und die dazugehörige Epoche x Hauptklasse aufgelistet.

In [6]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go

con = sqlite3.connect("Datenbank.db")

query = """SELECT epochen_genres_strata, autor, haupttitel FROM samples"""

df = pd.read_sql_query(query, con)
df_sorted = df.sort_values(by="epochen_genres_strata", ascending=True)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(df_sorted.columns),
        fill_color="paleturquoise",
        align="center"
    ),
    cells=dict(
        values=[df_sorted[col] for col in df_sorted.columns],
        fill_color="lavender",
        align="left"
    )
)])
con.close()
fig.update_layout(title="Textstichproben", height=3000)
fig.show()



Bei einer genaueren Betrachtung wird klar, dass die Epoche x Hauptklasse-Schicht Revolutionsjahre x Zeitung einen Bias gegenüber der Neuen Rheinischen Zeitung hat. Dieser Bias kann auch bestätigt werden, wenn die Gesamtanzahl aller Zeitungen aus den Revolutionsjahren miteinander verglichen werden.

In [7]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

con = sqlite3.connect("Datenbank.db")
query = """SELECT haupttitel FROM my_data WHERE epochen_genres_strata='Revolutionsjahre x Zeitung'"""

df = pd.read_sql_query(query, con)

haupttitel_count = df.groupby("haupttitel").size().reset_index(name="count")


fig = go.Figure(data=[go.Bar(name="Anzahl Texte pro Revolutionsjahre x Zeitung", x=haupttitel_count["haupttitel"], y=haupttitel_count["count"])])

fig.update_layout(
    title="Anzahl Texte pro Haupttitel in Revolutionsjahre x Zeitung",
    xaxis_title="Haupttitel",
    yaxis_title="Anzahl Texte",
    hovermode="x"
)

con.close()
fig.show()

Um ein gleichmässigeres Sampling zu gewährleisten werden die 15 Samples proportional aufgeteilt.

In [8]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()
query = """SELECT haupttitel FROM my_data WHERE epochen_genres_strata = 'Revolutionsjahre x Zeitung'"""

total_count = cur.execute("SELECT COUNT(epochen_genres_strata) FROM my_data WHERE epochen_genres_strata='Revolutionsjahre x Zeitung'").fetchone()[0]

df = pd.read_sql_query(query, con)

haupttitel_count = df.groupby("haupttitel").size().reset_index(name="count")


haupttitel_count["weight"] = haupttitel_count["count"] / total_count

haupttitel_count["sample_size"] = round(haupttitel_count["weight"] * 15, 0)

haupttitel_count["weight"] = round(haupttitel_count['weight'], 5)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(haupttitel_count.columns),
        fill_color="paleturquoise",
        align="center"
    ),
    cells=dict(
        values=[haupttitel_count[col] for col in haupttitel_count.columns],
        fill_color="lavender",
        align="left"
    )
)])

con.close()
fig.update_layout(title="Revolutionsjahre x Zeitung Aufschichtung")
fig.show()

Wie hier sichtbar wird, gibt es von den Zeitungen 
- Montag, 1. Mai
- Dienstag, 2. Mai
- Mittwoch, 3. Mai
- Donnerstag, 4. Mai
- Freitag, 5. Mai
- Allgemeine Auswandererzeitung Rudolstadt

nur jeweils ein Exemplar.

Es wird vermutet, dass die Zeitungen mit den Wochentagen, strukturell und linguistisch ähnlich sind. Zunächst wird überprüft, ob die Publikationsjahre übereinstimmen und der Herausgeber die gleiche Person ist.

In [9]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

outputs = cur.execute("""SELECT haupttitel, publikationsjahr, autor FROM my_data WHERE haupttitel IN (
                    'Montag, 1. Mai',
                    'Dienstag, 2. Mai',
                    'Mittwoch, 3. Mai',
                    'Donnerstag, 4. Mai',
                    'Freitag, 5. Mai'
                  )""").fetchall()

for output in outputs:
    title = output[0]
    year = output[1]
    author = output[2]
    print(f"Die Zeitschrift '{title}' wurde {year} von {author} publiziert.")

con.close()

Die Zeitschrift 'Montag, 1. Mai' wurde 1848 von Geyken, Alexander publiziert.
Die Zeitschrift 'Dienstag, 2. Mai' wurde 1848 von Geyken, Alexander publiziert.
Die Zeitschrift 'Mittwoch, 3. Mai' wurde 1848 von Geyken, Alexander publiziert.
Die Zeitschrift 'Donnerstag, 4. Mai' wurde 1848 von Geyken, Alexander publiziert.
Die Zeitschrift 'Freitag, 5. Mai' wurde 1848 von Geyken, Alexander publiziert.


Dies bestätigt die Annahme, dass alle Zeitungen im gleichen Jahr und vom gleichen Herausgeber sind. Der Vergleich der Zeitungen im Deutschen Textarchiv bestätigt zudem, dass sie zu der Zeitung "Staats und Gelehrte Zetung des Hamburgischen unpartheiischen Correspondenten" zugehörig sind:

https://www.deutschestextarchiv.de/book/view/hc_1040105_1848?p=1

https://www.deutschestextarchiv.de/book/view/hc_1050205_1848?p=1

https://www.deutschestextarchiv.de/book/view/hc_1060305_1848?p=1

https://www.deutschestextarchiv.de/book/view/hc_1070405_1848?p=1

https://www.deutschestextarchiv.de/book/view/hc_1080505_1848?p=1

Für den Preprocessing-Schritt reicht es deshalb, die Sample-Menge dieser Zeitung auf eins zu beschränken.

Nun wird geprüft, ob zwischen der Allgemeinen Auswandererzeitung Rudolstadt (ein Exemplar) und der Allgemeinen Auswanderungs-Zeitung (38 Exemplare) ein Zusammenhang besteht. Dabei wird Autorenschaft und Untertitel miteinander abgeglichen.

In [10]:
import sqlite3
import plotly.graph_objects as go

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

outputs = cur.execute("""SELECT DISTINCT haupttitel, untertitel, autor FROM my_data WHERE haupttitel IN ('Allgemeine Auswanderungs-Zeitung', 'Allgemeine Auswandererzeitung Rudolstadt')""").fetchall()

titles= [output[0] for output in outputs]
subtitles=[output[1] for output in outputs]
authors=[output[2] for output in outputs]

fig = go.Figure(data=[go.Table(header=dict(values=["Haupttitel", "Untertitel", "Autor"]), cells=dict(values=[titles, subtitles, authors]))])

fig.show()

con.close()


Hiermit ist klar ersichtlich, dass die Allgemeine Auswandererzeitung Rudolstadt die gleiche Zeitung darstellt wie die Allgemeine Auswanderungs-Zeitung. Daher kann diese weggellassen werden, ohne an Repräsentation zu verlieren.

Nun ergibt sich die Ajustierte Samplemenge wiefolgt: Von der Neuen Rheinischen Zeitung wird ein Sample weniger genommen, und von den fünf Wochentag-Zeitungen wird zufällig ein Sample herausgenommen. 

Die Samples werden neu gesammelt, mit der aktualisierten Strategie:

Zunächst werden alle Zufallsstichproben vom Typ "Revolutionsjahre x Zeitung" gelöscht

In [11]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

cur.execute("""DELETE FROM samples WHERE epochen_genres_strata='Revolutionsjahre x Zeitung'""")

con.commit()
con.close()

Danach wird ein Dictionary erstellt, das die Samplemenge an den jeweiligen Haupttitel anbindet.

In [12]:
import random
sample_list = {}

random_choice_of_mai = random.choice(['Montag, 1. Mai', 'Dienstag, 2. Mai', 'Mittwoch, 3. Mai', 'Donnerstag, 4. Mai','Freitag, 5. Mai'])

# Ajustieren der Samplemenge
haupttitel_count["adjusted_sample"] = [sample for sample in haupttitel_count["sample_size"]]
haupttitel_count.loc[haupttitel_count["haupttitel"] == random_choice_of_mai, "adjusted_sample"] += 1
haupttitel_count.loc[haupttitel_count["haupttitel"] == "Neue Rheinische Zeitung", "adjusted_sample"] -= 1

# Dictionary erstellen
samples = list(map(int, haupttitel_count.adjusted_sample))
titles = list(haupttitel_count.haupttitel)
sample_dict = dict(zip(titles, samples))





Stichproben sammeln

In [13]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

for title, sample_size in sample_dict.items():
    sample_size = int(sample_size)
    query = f"""
        INSERT INTO samples (text_index, haupttitel, untertitel, volumetitel, autor, hauptklasse, subklasse, sprache, text, publikationsjahr, zeitstratum, genrestratum, epochen_genres_strata)
                SELECT text_index, haupttitel, untertitel, volumetitel, autor, hauptklasse, subklasse, sprache, text, publikationsjahr, zeitstratum, genrestratum, epochen_genres_strata
                FROM my_data
                WHERE haupttitel = ?
                ORDER BY RANDOM()
                LIMIT {sample_size}

"""
    cur.execute(query, (title,))

con.commit()
con.close()

Nun kann überprüft werden, ob die aktualisierte Liste repräsentativer ist und weniger Wiederholungen enthält.

In [14]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go

con = sqlite3.connect("Datenbank.db")

query = """SELECT epochen_genres_strata, autor, haupttitel FROM samples"""

df = pd.read_sql_query(query, con)
df_sorted = df.sort_values(by="epochen_genres_strata", ascending=True)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(df_sorted.columns),
        fill_color="paleturquoise",
        align="center"
    ),
    cells=dict(
        values=[df_sorted[col] for col in df_sorted.columns],
        fill_color="lavender",
        align="left"
    )
)])
con.close()
fig.update_layout(title="Textstichproben", height=3000)
fig.show()



Und eine letzte Überprüfung, ob die Sample Zahl immer noch 100 ist:

In [15]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

print(cur.execute("SELECT COUNT(*) FROM samples").fetchone()[0])

100


### Da nun alle Stichproben gesammelt sind, ist dieses Jupyter Notebook hier zu Ende. In einem weiteren Schritt wird das Preprocessing anhand dieser 100 Stichproben erstellt ###